In [4]:
import cv2
import numpy as np
from ultralytics import YOLO
import argparse
from pathlib import Path

def detect_disk(model, image, disk_class=0, conf_thres=0.25):
    """
    ใช้โมเดล YOLO ใหม่ตรวจจับ disk ใน image
    คืน list ของ dict เหมือน bbox + center + class + score
    """
    results = model(image, conf=conf_thres)
    # results เป็น list (แต่เราส่ง image เดียว) → results[0]
    res = results[0]
    boxes = res.boxes  # กล่อง (x1, y1, x2, y2, confidence, cls)
    objs = []
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        cls = int(box.cls[0].cpu().numpy())
        conf = float(box.conf[0].cpu().numpy())
        w = x2 - x1
        h = y2 - y1
        cx = x1 + w/2
        cy = y1 + h/2
        objs.append({
            "class": cls,
            "conf": conf,
            "xmin": int(x1),
            "ymin": int(y1),
            "xmax": int(x2),
            "ymax": int(y2),
            "cx": int(cx),
            "cy": int(cy),
            "w": w,
            "h": h
        })
    return objs

def find_zone_of_inhibition(image, disk_center):
    """
    เหมือนเดิม — หา contour รอบ disk_center
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    blur = cv2.GaussianBlur(gray, (7,7), 0)
    edges = cv2.Canny(blur, 30, 100)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    edges = cv2.dilate(edges, kernel, iterations=1)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    cx, cy = disk_center
    candidates = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 50:
            continue
        inside = cv2.pointPolygonTest(cnt, (cx, cy), False)
        if inside >= 0:
            peri = cv2.arcLength(cnt, True)
            if peri == 0:
                continue
            circ = 4 * np.pi * area / (peri * peri)
            candidates.append((cnt, area, circ))
    if candidates:
        candidates.sort(key=lambda x: (x[2], x[1]), reverse=True)
        best = candidates[0][0]
    else:
        # fallback
        best = None
        for cnt in contours:
            (x,y), r = cv2.minEnclosingCircle(cnt)
            dist = np.hypot(x - cx, y - cy)
            if dist < 100:
                if best is None or cv2.contourArea(cnt) > cv2.contourArea(best):
                    best = cnt
        if best is None:
            return None, None
    (x, y), radius = cv2.minEnclosingCircle(best)
    return (int(x), int(y)), int(radius)

def measure_zone(image_path, model_path, real_disk_mm=6.0, disk_class=0, conf_thres=0.25, output_path="output.png"):
    """
    โหลด image, โหลด YOLO model รุ่นใหม่, ตรวจจับ disk, หา zone, วัด diameter (mm), บันทึกภาพผล
    """
    # โหลดภาพ
    image = cv2.imread(str(image_path))
    if image is None:
        raise FileNotFoundError(f"Cannot read image {image_path}")
    h, w = image.shape[:2]

    # โหลดโมเดล YOLO
    model = YOLO(model_path)  # เช่น "yolov11n.pt" หรือ path ของโมเดลคุณ

    # ตรวจจับ disk
    objs = detect_disk(model, image, disk_class, conf_thres)
    if not objs:
        raise RuntimeError("No disk detected")
    # เลือก object ที่เป็น disk (class) หรือ fallback
    disk_objs = [o for o in objs if o["class"] == disk_class]
    if not disk_objs:
        disk_objs = objs
    disk = max(disk_objs, key=lambda o: o["w"] * o["h"])
    cx, cy = disk["cx"], disk["cy"]
    disk_px_diameter = max(disk["w"], disk["h"])

    print(f"Disk detected center: ({cx},{cy}), pixel diameter ~ {disk_px_diameter:.1f}px")

    # หา zone
    zone_center, zone_radius = find_zone_of_inhibition(image, (cx, cy))
    if zone_radius is not None:
        zone_px_diameter = zone_radius * 2
        print(f"Zone pixel diameter ~ {zone_px_diameter:.1f}px")
    else:
        zone_px_diameter = None
        print("Zone not found")

    # แปลงเป็น mm
    px_per_mm = disk_px_diameter / real_disk_mm
    zone_mm = None
    if zone_px_diameter is not None:
        zone_mm = zone_px_diameter / px_per_mm
        print(f"Estimated zone diameter = {zone_mm:.2f} mm")

    # วาดผล
    vis = image.copy()
    cv2.circle(vis, (cx, cy), 3, (0,255,0), -1)
    if zone_center and zone_radius:
        cv2.circle(vis, zone_center, zone_radius, (0,0,255), 2)
        cv2.circle(vis, zone_center, 3, (0,0,255), -1)
    cv2.putText(vis, f"Zone: {zone_mm:.2f} mm" if zone_mm else "No zone", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    cv2.imwrite(output_path, vis)
    print(f"Visualization saved to {output_path}")

    return {
        "disk_center_px": (cx, cy),
        "disk_px_diameter": disk_px_diameter,
        "zone_px_diameter": zone_px_diameter,
        "zone_mm": zone_mm
    }

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--image", required=True, help="Path to input image")
    parser.add_argument("--model", required=True, help="Path to YOLO model (e.g. yolov11n.pt)")
    parser.add_argument("--real_disk_mm", type=float, default=6.0, help="Real disk diameter in mm")
    parser.add_argument("--disk_class", type=int, default=0, help="Class id for disk")
    parser.add_argument("--conf", type=float, default=0.25, help="Confidence threshold")
    parser.add_argument("--output", default="output.png", help="Output visualization image")
    args = parser.parse_args()

    res = measure_zone(args.image, args.model, args.real_disk_mm, args.disk_class, args.conf, args.output)
    print("Result:", res)


usage: ipykernel_launcher.py [-h] --image IMAGE --model MODEL
                             [--real_disk_mm REAL_DISK_MM]
                             [--disk_class DISK_CLASS] [--conf CONF]
                             [--output OUTPUT]
ipykernel_launcher.py: error: the following arguments are required: --image, --model


AttributeError: 'tuple' object has no attribute 'tb_frame'